# PROJECT 3: Startup Financial KPI Dashboard

#### PROJECT GOAL: 
Build a startup financial performance model to track burn rate, runway, unit economics, and growth KPIs, using SQL-first analysis and an executive Power BI dashboard.

1. ENVIRONMENT SETUP

In [4]:
import sys
sys.executable

'c:\\Users\\Zouitina Rihab\\Desktop\\finance proj3\\.venv\\Scripts\\python.exe'

In [5]:
!pip install duckdb pandas numpy matplotlib seaborn

In [6]:
import duckdb
duckdb.__version__

'1.4.3'

2. Data generation

We generate synthetic but realistic startup financial data to simulate revenue, expenses, and headcount evolution.

In [7]:
import duckdb
import pandas as pd
import numpy as np

np.random.seed(42)

dates = pd.date_range("2023-01-01", "2024-12-31", freq="M")

transactions = []
for date in dates:
    # Revenue
    transactions.append([date, "revenue", "subscriptions", np.random.randint(80000, 130000), "Sales"])
    
    # Expenses
    transactions.append([date, "expense", "salaries", np.random.randint(50000, 80000), "HR"])
    transactions.append([date, "expense", "marketing", np.random.randint(15000, 30000), "Marketing"])
    transactions.append([date, "expense", "infrastructure", np.random.randint(8000, 15000), "Tech"])

transactions_df = pd.DataFrame(
    transactions,
    columns=["date", "type", "category", "amount", "department"]
)

transactions_df.head()


C:\Users\Zouitina Rihab\AppData\Local\Temp\ipykernel_17648\2963083027.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range("2023-01-01", "2024-12-31", freq="M")


,date,type,category,amount,department
0,2023-01-31,revenue,subscriptions,95795,Sales
1,2023-01-31,expense,salaries,50860,HR
2,2023-01-31,expense,marketing,20390,Marketing
3,2023-01-31,expense,infrastructure,13226,Tech
4,2023-02-28,revenue,subscriptions,124732,Sales


LOADING INTO DUCKDB

In [8]:
con = duckdb.connect(database="startup_finance.duckdb")

con.execute("CREATE OR REPLACE TABLE transactions AS SELECT * FROM transactions_df")

con.execute("SELECT COUNT(*) FROM transactions").fetchall()

[(96,)]

– Monthly Revenue : calculates monthly revenue from subscriptions to track growth over time.


In [9]:
revenue_df = con.execute("""
SELECT 
    date,
    SUM(amount) AS monthly_revenue
FROM transactions
WHERE type = 'revenue'
GROUP BY date
ORDER BY date
""").df()

revenue_df.head()


,date,monthly_revenue
0,2023-01-31,95795.0
1,2023-02-28,124732.0
2,2023-03-31,96850.0
3,2023-04-30,127191.0
4,2023-05-31,96023.0


– Monthly Expenses: aggregates all operational expenses to monitor cost structure.


In [10]:
expenses_df = con.execute("""
SELECT 
    date,
    SUM(amount) AS monthly_expenses
FROM transactions
WHERE type = 'expense'
GROUP BY date
ORDER BY date
""").df()

expenses_df.head()

,date,monthly_expenses
0,2023-01-31,84476.0
1,2023-02-28,96283.0
2,2023-03-31,112914.0
3,2023-04-30,115302.0
4,2023-05-31,83776.0


– Burn Rate: represents how much cash the startup is losing (or gaining) each month.

In [11]:
burn_rate_df = con.execute("""
SELECT 
    date,
    SUM(CASE WHEN type = 'expense' THEN amount ELSE 0 END) -
    SUM(CASE WHEN type = 'revenue' THEN amount ELSE 0 END) AS burn_rate
FROM transactions
GROUP BY date
ORDER BY date
""").df()

burn_rate_df.head()

,date,burn_rate
0,2023-01-31,-11319.0
1,2023-02-28,-28449.0
2,2023-03-31,16064.0
3,2023-04-30,-11889.0
4,2023-05-31,-12247.0


– Expense Breakdown by Category: to analyze cost drivers such as salaries, marketing, and infrastructure.

In [12]:
expense_breakdown_df = con.execute("""
SELECT 
    category,
    SUM(amount) AS total_expense
FROM transactions
WHERE type = 'expense'
GROUP BY category
ORDER BY total_expense DESC
""").df()

expense_breakdown_df

,category,total_expense
0,salaries,1557247.0
1,marketing,553479.0
2,infrastructure,270127.0


– Export Financial KPIs for Power BI: The dataset below is used to build the executive dashboard.


In [13]:
bi_df = con.execute("""
SELECT 
    date,
    SUM(CASE WHEN type='revenue' THEN amount ELSE 0 END) AS revenue,
    SUM(CASE WHEN type='expense' THEN amount ELSE 0 END) AS expenses,
    SUM(CASE WHEN type='expense' THEN amount ELSE 0 END) -
    SUM(CASE WHEN type='revenue' THEN amount ELSE 0 END) AS burn_rate
FROM transactions
GROUP BY date
ORDER BY date
""").df()

bi_df.to_csv("startup_financial_kpis.csv", index=False)
bi_df.head()

,date,revenue,expenses,burn_rate
0,2023-01-31,95795.0,84476.0,-11319.0
1,2023-02-28,124732.0,96283.0,-28449.0
2,2023-03-31,96850.0,112914.0,16064.0
3,2023-04-30,127191.0,115302.0,-11889.0
4,2023-05-31,96023.0,83776.0,-12247.0


In [17]:
table_df = con.execute("""SELECT
    date,
    category,
    department,
    amount
FROM transactions
WHERE type = 'expense'
""").df()

table_df.to_csv("expenses_by_category.csv", index=False)
table_df.head()

,date,category,department,amount
0,2023-01-31,salaries,HR,50860
1,2023-01-31,marketing,Marketing,20390
2,2023-01-31,infrastructure,Tech,13226
3,2023-02-28,salaries,HR,61284
4,2023-02-28,marketing,Marketing,20734


In [18]:
table2_df = con.execute("""SELECT
    date,
    department,
    amount
FROM transactions
WHERE type = 'revenue'
""").df()

table2_df.to_csv("revenue_by_department.csv", index=False)
table2_df.head()

,date,department,amount
0,2023-01-31,Sales,95795
1,2023-02-28,Sales,124732
2,2023-03-31,Sales,96850
3,2023-04-30,Sales,127191
4,2023-05-31,Sales,96023
